In [24]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import metpy
import numbers



In [ ]:
# read in data
data_location = 'data'
outlooks, pph, reports = read_datasets(data_location, 'labelled')

In [4]:
def consolidate_date(outlooks, pph, reports):
    earliest_date = max(min(outlooks['DATE']), min(pph['time']), min(reports['DATE'])) #TODO first two days of outlook dataset don't have day 3 forecast. but probably ok since not mdt
    latest_date = min(max(outlooks['DATE']), max(pph['time']), max(reports['DATE']))
    reports = reports[reports['DATE'] <= latest_date]
    reports = reports[reports['DATE'] >= earliest_date]
    outlooks = outlooks[outlooks['DATE'] <= latest_date]
    outlooks = outlooks[outlooks['DATE'] >= earliest_date]
    all_pph_dates = pph['time']
    pph_dates = all_pph_dates[all_pph_dates <= latest_date]
    pph_dates = pph_dates[pph_dates >= earliest_date]
    pph = pph.sel(time = pph_dates)
    return(outlooks, pph, reports)

(outlooks, pph, reports) = consolidate_date(outlooks, pph, reports)


In [3]:
label_names = ['MAX_CAT', 'RAMP_CAT', 'SEASON', 'REGION_M', 'RAMP_UP', 'RAMP_DOWN', 
               'PPH_D_NUM',
               'REPORT_NUM', 'TOR_NUM', 'WIND_NUM', 'HAIL_NUM', 
               'TOR_F', 'WINDSP_NUM', 'HAILSZ_NUM', 
               'BS_NUM', 'NEIGH_NUM', 'RMSE_NUM',
               'POD_NUM', 'FAR_NUM',
               'POD_H_NUM', 'FAR_H_NUM',
               'POD_W_NUM', 'FAR_W_NUM',
               'POD_T_NUM', 'FAR_T_NUM',
               'FART_NUM',
               'E_SH_NUM', 'E_SH_H_NUM', 'E_SH_W_NUM', 'E_SH_T_NUM',
               'N_SH_NUM', 'N_SH_H_NUM', 'N_SH_W_NUM', 'N_SH_T_NUM',
               'DIV_NUM', 'DIV_H_NUM', 'DIV_W_NUM', 'DIV_T_NUM'
               ]

category_dict = {
    'NONE' : -1,
    'TSTM': 0,
    'MRGL': 1,
    'SLGT': 2,
    'ENH': 3,
    'MDT': 4,
    'HIGH': 5
}

ramp_dict = {
    'up': 0,
    'down': 1,
    'both': 2,
    'neither': 3
}

season_dict = {
    'Winter': 0,
    'Spring': 1,
    'Summer': 2,
    'Fall': 3
}

region_dict = {
    'NONE': -1,
    'West': 0,
    'Great Plains': 1,
    'Midwest': 2,
    'Northeast': 3,
    'South': 4
}

ramp_up_dict = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6
}

ramp_down_dict = {
    '0': 0,
    '-1': 1,
    '-2': 2,
    '-3': 3,
    '-4': 4,
    '-5': 5,
    '-6': 6
}

pph_dict = {
    'ZERO': 0,
    'TSTM': 1,
    'MRGL': 2,
    'SLGT': 3,
    'ENH': 4,
    'MDT': 5,
    'HIGH': 6
}

tor_dict = {
    'NONE': -1,
    'EFU': 0,
    '(E)F0': 1,
    '(E)F1': 2,
    '(E)F2': 3,
    '(E)F3': 4,
    '(E)F4': 5,
    '(E)F5': 6
}

dicts = [category_dict, ramp_dict, season_dict, region_dict, ramp_up_dict, ramp_down_dict, 
         None, 
         None, None, None, None, 
         tor_dict, None, None,
         None, None, None,
         None, None,
         None, None,
         None, None,
         None, None,
         None,
         None, None, None, None,
         None, None, None, None,
         None, None, None, None] 

steps = [1, 1, 1, 1, 1, 1, 
         10, 
         100, 25, 100, 50, 
         1, 10, 1, 
         .0025, .001, .01,
         .05, .1, 
         .05, .1,
         .05, .1,
         .05, .1,
         .001,
         100000, 100000, 100000, 100000,
         100000, 100000, 100000, 100000,
         .5, .5, .5, .5
         ]

written_labels = ['Categorical Risk', 'Ramp', 'Season', 'Region', 'Ramp Up', 'Ramp Down', 
                  'Max PPH',
                  'Total Storm Reports', 'Tornado Reports', 'Wind Reports', 'Hail Reports',
                  'Max Tornado Rating', 'Max Wind Speed (kt)', 'Max Hail Size (in)',
                  'Brier Score', 'Neighborhood Brier Score', 'RMSE',
                  'Probability of Detection', 'False Alarm Ratio',
                  'Hail Probability of Detection', 'Hail False Alarm Ratio',
                  'Wind Probability of Detection', 'Wind False Alarm Ratio',
                  'Tornado Probability of Detection', 'Tornado False Alarm Ratio',
                  'False Alarm Rate',
                  'East Shift', 'Hail East Shift', 'Wind East Shift', 'Tornado East Shift',
                  'North Shift', 'Hail North Shift', 'Wind North Shift', 'Tornado North Shift',
                  'Divergence', 'Hail Divergence', 'Wind Divergence', 'Tornado Divergence'
                  ]


In [4]:
def plot_distribution(pph, label_name, label_dict, written_label, titlestring, save_location, bins = 20, show=False):
    if label_dict == None:
        plt.hist(pph[label_name].values, bins = bins)
    else:
        labels = pph[label_name]
        label_counts = labels.groupby(labels).count()
        def sort_order(key):
            return label_dict[str(key[0])]
        sorted_labels, sorted_counts = zip(*sorted(zip(list(label_counts[label_name].values), label_counts.values), key = sort_order))
        plt.bar(sorted_labels, sorted_counts)  
    plt.xlabel(written_label)
    plt.title('Number of ' + titlestring + ' Days With Each '  + written_label)
    plt.ylabel("Number of Days")
    if save_location != None:
        plt.savefig(save_location + '/' + written_label + '_distribution.png')
    if show:
        plt.show()
    plt.close()

In [ ]:
plot_distribution(pph, 'E_SH_NUM', None, 'East Shift', '', None, show = True)

In [6]:
def plot_distribution_2d(pph, label_1, label_2, label_1_string, label_2_string, dict_1, dict_2, titlestring, save_location, show=False, defaultbins = 10, step_1 = 1, step_2 = 1):
# plot 2d heatmap for any 2 labels

    data1 = []
    data2 = []

    for i in range(len(pph[label_1])):
        if dict_1 != None:
            data1.append(dict_1[str(pph[label_1].values[i])])
        else: 
            data1.append(pph[label_1].values[i])
        if dict_2 != None:
            data2.append(dict_2[str(pph[label_2].values[i])])
        else:
            data2.append(pph[label_2].values[i])

    
    if dict_1 != None:
        max1 = max(dict_1.values())
        min1 = min(dict_1.values())
        bins1 = np.linspace(min1-.5, max1+.5, 2+max1-min1)
        irange = bins1.size-1

    else:
        if ((max(data1) - min(data1))/step_1).is_integer():
            m = 2
        else: 
            m = 1
            
        if min(data1) < 0:
            bins1 = np.arange(step_1 * round(min(data1)/step_1), max(data1)+m*step_1, step_1)
        else:
            bins1 = np.insert(np.arange(step_1 * round(min(data1)/step_1), max(data1)+m*step_1, step_1), 0, 0)
        
            if isinstance(data1[1], numbers.Integral):
                bins1[1] = bins1[1] + 1
            else:
                bins1[1] = bins1[1] + .00000000001
        num_bins1 = len(bins1)-1
        irange = num_bins1

    if dict_2 != None:
        max2 = max(dict_2.values())
        min2 = min(dict_2.values())
        bins2 = np.linspace(min2-.5, max2+.5, 2+max2-min2)
        jrange = bins2.size-1
    else:
        if ((max(data2) - min(data2))/step_2).is_integer():
            m = 2
        else: 
            m = 1

        if min(data2) < 0:
            bins2 = np.arange(step_2 * round(min(data2)/step_2), max(data2)+m*step_2, step_2)
        else:
            bins2 = np.insert(np.arange(step_2 * round(min(data2)/step_2), max(data2)+m*step_2, step_2), 0, 0)

            if isinstance(data2[1], numbers.Integral):
                bins2[1] = bins2[1] + 1
            else:
                bins2[1] = bins2[1] + .00000000001
        num_bins2 = len(bins2)-1
        jrange = num_bins2

    heatmap = np.histogram2d(data1, data2, bins = (bins1, bins2))
    im = plt.imshow(heatmap[0],  norm=colors.LogNorm())
    plt.colorbar(im)

    for i in range(irange): 
        for j in range(jrange): 
            plt.annotate(str(int(heatmap[0][i][j])), xy=(j, i), 
                        ha='center', va='center', color='black') 

    if dict_1 != None:
        plt.yticks(range(len(list(dict_1.values()))), labels=list(dict_1.keys()))
    else:
        labels = [s + '+' for s in heatmap[1][range(num_bins1)].astype(str)]
        if min(data1) >= 0:
            labels[0] = '0'
            if isinstance(data1[1], numbers.Integral):
                labels[1] = '1+'
            else:
                labels[1] = '0+'
        plt.yticks(range(num_bins1), labels = labels) 
        plt.yticks(fontsize=8)
    if dict_2 != None:
        plt.xticks(range(len(list(dict_2.values()))), labels=list(dict_2.keys()))
    else:
        labels = [s + '+' for s in heatmap[2][range(num_bins2)].astype(str)]
        if min(data2) >= 0:
            labels[0] = '0'
            if isinstance(data2[1], numbers.Integral):
                labels[1] = '1+'
            else:
                labels[1] = '0+'
        plt.xticks(range(num_bins2), labels = labels)
        plt.xticks(fontsize=8)

    plt.ylabel(label_1_string)
    plt.xlabel(label_2_string)

    plt.title("Number of " + titlestring + " Days with each Combination of " + label_1_string + " and " +  label_2_string)
    if save_location != None:
        plt.savefig(save_location + '/' + label_1_string + '_' + label_2_string + '_distribution.png')
    if show:
        plt.show()
    plt.close()

In [ ]:
plot_distribution_2d(pph, 'N_SH_NUM', 'E_SH_NUM', 'North Shift', 'East Shift', None, None, '', None, show = True, step_1 = 100000, step_2 = 100000)

In [5]:
# This makes the next steps work properly for some reason
for k in list(pph.keys()):
    pph[k].values

# moderate and up days only:
all_pph_dates = pph['time']
mdt_pph_dates = all_pph_dates[pph['MAX_CAT'].isin(['MDT', 'HIGH'])]
mdt_pph = pph.sel(time = mdt_pph_dates)

# dates since new categorical system 
new_cutoff = '201410230000'
new_pph_dates = all_pph_dates[all_pph_dates >= new_cutoff]
new_pph = pph.sel(time = new_pph_dates)

# Moderate dates in new system
mdt_new_pph_dates = mdt_pph_dates[mdt_pph_dates >= new_cutoff]
mdt_new_pph = pph.sel(time = mdt_new_pph_dates)


# dates since day 3 added
day3_cutoff = '200203300000'
day3_pph_dates = all_pph_dates[all_pph_dates >= day3_cutoff]
day3_pph = pph.sel(time = day3_pph_dates)

# Moderate dates since day 3 added
mdt_day3_pph_dates = mdt_pph_dates[mdt_pph_dates >= day3_cutoff]
mdt_day3_pph = pph.sel(time = mdt_day3_pph_dates)


# dates since early day 2 added
day21_cutoff = '199707100000'
day21_pph_dates = all_pph_dates[all_pph_dates >= day21_cutoff]
day21_pph = pph.sel(time = day21_pph_dates)


# Moderate dates since early day 2 added
mdt_day21_pph_dates = mdt_pph_dates[mdt_pph_dates >= day21_cutoff]
mdt_day21_pph = pph.sel(time = mdt_day21_pph_dates)


# dates since late day 2 added
day22_cutoff = '199504040000'
day22_pph_dates = all_pph_dates[all_pph_dates >= day22_cutoff]
day22_pph = pph.sel(time = day22_pph_dates)

# Moderate since late day 2 added
mdt_day22_pph_dates = mdt_pph_dates[mdt_pph_dates >= day22_cutoff]
mdt_day22_pph = pph.sel(time = mdt_day22_pph_dates)



In [9]:
def make_plots(pph, label_names, dicts, written_labels, shortstring, titlestring, show=False):
    
    for i in range(len(written_labels)):
        plot_distribution(pph, label_names[i], dicts[i], written_labels[i], titlestring, 'plots/label_distributions/'+ shortstring+'/1d/', show=show)

    for i in range(len(label_names)):
        for j in range(i):
            plot_distribution_2d(pph, label_names[i], label_names[j], written_labels[i], written_labels[j], dicts[i], dicts[j], titlestring, 'plots/label_distributions/'+ shortstring+'/2d/', show=show, step_1 = steps[i], step_2 = steps[j])

In [10]:
#make_plots(pph, label_names, dicts, written_labels, 'all', 'All')
#make_plots(mdt_pph, label_names, dicts, written_labels, 'mdt', 'Moderate')

#make_plots(new_pph, label_names, dicts, written_labels, 'new', 'New')
#make_plots(mdt_new_pph, label_names, dicts, written_labels, 'mdt_new', 'Moderate New')

make_plots(day3_pph, label_names, dicts, written_labels, 'day3', 'Since 2002')
make_plots(mdt_day3_pph, label_names, dicts, written_labels, 'mdt_day3', 'Moderate Since 2002')

#make_plots(day21_pph, label_names, dicts, written_labels, 'day21', 'Since 1997')
#make_plots(mdt_day21_pph, label_names, dicts, written_labels, 'mdt_day21', 'Moderate Since 1997')

#make_plots(day22_pph, label_names, dicts, written_labels, 'day22', 'Since 1995')
#make_plots(mdt_day22_pph, label_names, dicts, written_labels, 'mdt_day22', 'Moderate Since 1995')


# Timeseries

In [11]:
def plot_timeseries_2(pph, vars_of_interest, var_names, save = True, save_location = 'plots/timeseries', show = True):
    p = pph.sel(time = slice('200203300000', '202212310000'))
    times = pd.to_datetime(p['time'].astype(str), format='%Y%m%d%H%M')

    rolling = 365
    half_rolling_1 = int(np.floor(rolling/2))
    half_rolling_2 = rolling - half_rolling_1 - 1

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    ax1.plot(times[half_rolling_1:-half_rolling_2], np.convolve(p[vars_of_interest[0]].data, np.ones(rolling)/rolling, 'valid'))
    ax2.plot(times[half_rolling_1:-half_rolling_2], np.convolve(p[vars_of_interest[1]].data, np.ones(rolling)/rolling, 'valid'), color = 'r')
    ax1.set_ylabel(var_names[0])
    ax2.set_ylabel(var_names[1])
    ax1.set_xlabel('Centered Time')
    plt.title(var_names[0] + ' and ' + var_names[1] + ' 1-Year Running Mean Since 2002')

    fig.tight_layout() 
    fig.legend(var_names)

    if save:
        plt.savefig(save_location + '/timeseries_' + var_names[0] + var_names[1] + '.png')
    if show:
        plt.show()

def plot_timeseries_n(pph, vars_of_interest, var_names, mod_name, ylabel, save = True, save_location = 'plots/timeseries', show = True):
    p = pph.sel(time = slice('200203300000', '202212310000'))
    times = pd.to_datetime(p['time'].astype(str), format='%Y%m%d%H%M')

    rolling = 365
    half_rolling_1 = int(np.floor(rolling/2))
    half_rolling_2 = rolling - half_rolling_1 - 1

    for var, name in zip(vars_of_interest, var_names):
        plt.plot(times[half_rolling_1:-half_rolling_2], np.convolve(p[var].data, np.ones(rolling)/rolling, 'valid'))
    
    plt.xlabel('Centered Time')
    plt.ylabel(ylabel)
    plt.title(ylabel + ' 1-Year Running Mean Since 2002')
    plt.tight_layout() 
    plt.legend(var_names)
    
    if save:
        plt.savefig(save_location + '/timeseries_' + mod_name + '.png')

    if show:
        plt.show()



In [ ]:
var_lists_2 = [['POD_NUM', 'FAR_NUM'],
             ['RAMP_UP', 'RAMP_DOWN'],
             ['FAR_NUM', 'FART_NUM'],
             ['POD_W_NUM', 'FAR_W_NUM'],
             ['POD_H_NUM', 'FAR_H_NUM'],
             ['POD_T_NUM', 'FAR_T_NUM']]

name_lists_2 = [['Probability of Detection', 'False Alarm Ratio'],
             ['Ramp Up', 'Ramp Down'],
             ['False Alarm Ratio', 'False Alarm Rate'],
             ['Wind POD', 'Wind FAR'],
             ['Hail POD', 'Hail FAR'],
             ['Tornado POD', 'Tornado FAR']]

for var, names in zip(var_lists_2, name_lists_2):
    plot_timeseries_2(pph, var, names)

In [ ]:
var_lists_n = [['REPORT_NUM', 'TOR_NUM', 'WIND_NUM', 'HAIL_NUM'],
               ['PPH_D_NUM', 'PPH_NUM'],
               ['POD_W_NUM', 'POD_H_NUM', 'POD_T_NUM'],
               ['FAR_W_NUM', 'FAR_H_NUM', 'FAR_T_NUM'],
               ['DIV_NUM', 'DIV_W_NUM', 'DIV_H_NUM', 'DIV_T_NUM'],
               ['E_SH_NUM', 'E_SH_W_NUM', 'E_SH_H_NUM', 'E_SH_T_NUM'],
               ['N_SH_NUM', 'N_SH_W_NUM', 'N_SH_H_NUM', 'N_SH_T_NUM']
]

name_lists_n = [['Total', 'Tornado', 'Wind', 'Hail'],
                ['Dependent Hazards', 'Independent Hazards'],
                ['Wind POD', 'Hail POD', 'Tornado POD'],
                ['Wind FAR', 'Hail FAR', 'Tornado FAR'],
                ['Divergence', 'Wind Divergence', 'Hail Divergence', 'Tornado Divergence'],
                ['East Shift', 'Wind East Shift', 'Hail East Shift', 'Tornado East Shift'],
                ['North Shift', 'Wind North Shift', 'Hail North Shift', 'Tornado North Shift']
    
]

mod_names = ['reports', 'pph', 'pod', 'far', 'div', 'e_shift', 'n_shift']

axis_names = ['Daily Storm Reports', 'Daily Max PPH', 'Probability of Detection', 'False Alarm Ratio', 'Divergence', 'East Shift (m)', 'North Shift (m)']

for var, names, mod_name, axis_name in zip(var_lists_n, name_lists_n, mod_names, axis_names):
    plot_timeseries_n(pph, var, names, mod_name, axis_name)

In [ ]:
grid_outlook_location = 'data/outlooks/grid_outlooks.nc'
grid_outlooks = xr.open_dataset(grid_outlook_location)
g = grid_outlooks.sel(time = slice('200203300000', '202212310000'))
times = pd.to_datetime(g['time'].astype(str), format='%Y%m%d%H%M')
rolling = 365
half_rolling_1 = int(np.floor(rolling/2))
half_rolling_2 = rolling - half_rolling_1 - 1

In [ ]:
plt.plot(times[half_rolling_1:-half_rolling_2], np.convolve(g['prob'].sel(outlook = 'Day 1').max(dim=['x', 'y']).data, np.ones(rolling)/rolling, 'valid'))
plt.plot(times[half_rolling_1:-half_rolling_2], np.convolve(g['prob'].sel(outlook = 'Day 2 17').max(dim=['x', 'y']).data, np.ones(rolling)/rolling, 'valid'))
plt.plot(times[half_rolling_1:-half_rolling_2], np.convolve(g['prob'].sel(outlook = 'Day 2 7').max(dim=['x', 'y']).data, np.ones(rolling)/rolling, 'valid'))
plt.plot(times[half_rolling_1:-half_rolling_2], np.convolve(g['prob'].sel(outlook = 'Day 3').max(dim=['x', 'y']).data, np.ones(rolling)/rolling, 'valid'))
plt.legend(['Day 1', 'Day 2 17z', 'Day 2 7z', 'Day 3'])

plt.title('Daily Maximum Any-Hazard Probability Over Time')

plt.ylabel('1-year Running Mean Daily Maximum Any-Hazard Probability')
plt.xlabel('Centered Time')    
plt.savefig('plots/timeseries/prob_over_time.png')


# Displacement Histograms

In [ ]:
# TODO: Custom colors and label order (south and midwest on bottom, seasons in order)
def stacked_histogram(pph, var_of_interest, var_stack, xlabel, titlestring, legend_title, save_location, bins = 40, show=False, save = True):
    
    plot_arrays = []

    all_var_of_interest = pph[var_of_interest]

    all_var_stack = pph[var_stack]
    stack_values = list(set(all_var_stack.values))

    for stack_value in stack_values:

        plot_arrays.append(all_var_of_interest[all_var_stack == stack_value]/1000)

    cmap = plt.get_cmap('Purples')
    colors = [cmap(.5 + i / 2 / len(plot_arrays)) for i in range(len(plot_arrays))]

    plt.hist(plot_arrays, bins, stacked=True, label = stack_values, color = colors, range = (-800, 800))
    plt.plot([0, 0], [0, 50], color='k', linestyle='-')
    plt.xlabel(xlabel)
    plt.title(titlestring)
    plt.legend(title = legend_title)

    if save:
        plt.savefig(save_location + '.png')
    if show:
        plt.show()



stacked_histogram(mdt_day3_pph, 'E_SH_W_NUM', 'REGION_M', 'Eastward Displacement (km)', 'Wind Eastward Optical Flow Displacement on MDT+ Days 2003-2022, by Region', 'Region', 'plots/results/wind_displacement.png', show = True, save = False)
stacked_histogram(mdt_day3_pph, 'E_SH_H_NUM', 'SEASON', 'Eastward Displacement (km)', 'Hail Eastward Optical Flow Displacement on MDT+ Days 2003-2022, by Season', 'Season', 'plots/results/hail_displacement.png', show = True, save = False)

# OLD NON-GENERALIZED PLOTTING OF RAMPS / SCRATCH WORK

Investigate: old ramp ups by 5? Why lots of 5 and not 4? And none region, make sure no PPH.

In [ ]:
test = outlooks[outlooks['REGION'] != 'NONE']
test[test['DATE'] <= '199203160000']

In [ ]:
test_outlooks = outlooks[outlooks['MAX_CAT'] == 'SLGT']
test_outlooks = test_outlooks[test_outlooks['RAMP_UP'] == 3]
set(test_outlooks['DATE'])

In [ ]:
outlooks[outlooks['DATE'] == '200208030000']

In [ ]:
# TODO: make function that plots all 3 outlooks, PPH, and reports for one day to spotcheck
# with utils_plotting functions
def plot_day(datestring, outlooks, pph, reports):
    outlooks_day = outlooks[outlooks['DATE'] == datestring]
    pph_day = pph.sel(time=datestring)
    reports_day = reports[reports['DATE'] == datestring]
    
    print('plotting outlooks')
    plot_outlooks_day(outlooks_day, 'plots/daily/'+datestring+'/outlooks', ['CATEGORICAL'], show=True)
    print('plotting pph')
    plot_pph_day(pph_day, 'plots/daily/'+datestring+'/pph', ['total'], show=True, sig = False)
    #print('plotting reports')
    #plot_reports(reports_day, 'plots/daily/'+datestring+'/reports', ['Hail', 'Thunderstorm Wind', 'Tornado'], show=True)
    return

#plot_day('200504220000', outlooks, pph, reports)

In [ ]:
plot_day('201010100000', outlooks, pph, reports)

In [ ]:
def plot_ramps(pph, title_insert=''):
    # 2d histogram of ramps
    # NOT FOR USE
    ramp_up_bins = [-.5, .5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5]
    ramp_up_amounts = [0, 1, 2, 3, 4, 5, 6]
    ramp_down_amounts = [-6, -5, -4, -3, -2, -1, 0]
    ramp_down_bins = [-6.5, -5.5, -4.5, -3.5, -2.5, -1.5, -.5, .5]
    heatmap = np.histogram2d(np.array(pph['RAMP_UP'][pph['RAMP_UP']!= 'NONE'], dtype=int), np.array(pph['RAMP_DOWN'][pph['RAMP_UP']!= 'NONE'], dtype=int), bins = (ramp_up_bins, ramp_down_bins))
    im = plt.imshow(heatmap[0],  norm=colors.LogNorm())
    plt.colorbar(im)
    bins = len(ramp_up_bins)
    for i in range(bins-1): 
        for j in range(bins-1): 
            plt.annotate(str(int(heatmap[0][i][j])), xy=(j, i), 
                        ha='center', va='center', color='black') 

    plt.xticks(ramp_up_amounts, labels=ramp_down_amounts)
    plt.yticks(ramp_up_amounts, labels=ramp_up_amounts)
    plt.xlabel("Ramp Down")
    plt.ylabel("Ramp Up")
    plt.title("Number of " + title_insert + "Days with each Ramp Up and Ramp Down Amount")
    plt.show()

In [ ]:
str(mdt_pph['time'].values).replace('\n ', ' ').replace(' ', ', ')

In [ ]:
mdt_new_pph['time'].where(mdt_new_pph['MAX_CAT'] == 'HIGH')
